In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", 
                               filename="ted_en-20160408.zip")


In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)


In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


## Part 2: Word Frequencies

In [14]:
from collections import Iterable

def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

In [15]:
words = [word for word in flatten(sentences_ted)]

In [18]:
import collections
word_counter=collections.Counter(words)

In [30]:
aa = word_counter.most_common()
counts_ted_top1000 = aa[:1000]


In [32]:
top1000_val = [v[1] for v in counts_ted_top1000]

In [34]:
hist, edges = np.histogram(top1000_val, density=True, bins=100, normed=True)

In [35]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [36]:
from gensim.models import Word2Vec

Using TensorFlow backend.


In [37]:
model = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

In [38]:
model.most_similar("man")

[('woman', 0.8460798263549805),
 ('guy', 0.8102425336837769),
 ('lady', 0.7796527147293091),
 ('girl', 0.7529878616333008),
 ('boy', 0.7444412112236023),
 ('gentleman', 0.7003335952758789),
 ('poet', 0.6951100826263428),
 ('david', 0.6939632296562195),
 ('kid', 0.6897885799407959),
 ('soldier', 0.6809227466583252)]

In [40]:
model.most_similar("computer")

[('machine', 0.7574893832206726),
 ('software', 0.7415962815284729),
 ('robot', 0.6805332899093628),
 ('device', 0.6590874195098877),
 ('3d', 0.655287504196167),
 ('mechanical', 0.6357251405715942),
 ('program', 0.6354331970214844),
 ('camera', 0.6281588077545166),
 ('interface', 0.6258364915847778),
 ('simulation', 0.616331934928894)]

## t-SNE visualization with D3.js

In [46]:

top1000_word = [word[0] for word in counts_ted_top1000]
print(top1000_word)

['the', 'and', 'to', 'of', 'a', 'that', 'i', 'in', 'it', 'you', 'we', 'is', 's', 'this', 'so', 'they', 'was', 'for', 'are', 'have', 'but', 'what', 'on', 'with', 'can', 't', 'about', 'there', 'be', 'as', 'at', 'all', 'not', 'do', 'my', 'one', 're', 'people', 'like', 'if', 'from', 'now', 'our', 'he', 'an', 'just', 'these', 'or', 'when', 'because', 'very', 'me', 'out', 'by', 'them', 'how', 'know', 'up', 'going', 'had', 'more', 'think', 'who', 'were', 'see', 'your', 'their', 'which', 'would', 'here', 'really', 'get', 've', 'then', 'm', 'world', 'us', 'time', 'some', 'has', 'don', 'actually', 'into', 'way', 'where', 'will', 'years', 'things', 'other', 'no', 'could', 'go', 'well', 'want', 'been', 'make', 'right', 'she', 'said', 'something', 'those', 'first', 'two', 'than', 'much', 'also', 'look', 'new', 'thing', 'little', 'got', 'back', 'over', 'most', 'say', 'even', 'his', 'life', 'only', 'work', 'many', 'take', 'need', 'did', 'lot', 'kind', 'why', 'good', 'around', 'every', 'different', 'd

In [48]:

words_top_vec_wiki = model[top1000_word]
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [50]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=top1000_word))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)